### Givens rotation

#### Triangularization using Givens rotation

1. Settings

In [54]:
import numpy as np

# 4-by-4 example
n = 4

# creat a quick, full rank matrix without typing
tmp = np.arange(1, n+1, dtype=np.float64)

# Creat A and duplicate it (A changes along the line)
A = tmp.reshape(-1,1) ** tmp
A_ini = A.copy()

print(A)


[[  1.   1.   1.   1.]
 [  2.   4.   8.  16.]
 [  3.   9.  27.  81.]
 [  4.  16.  64. 256.]]


2. Zero out (2,1) entry to begin with


2.A. Submatrix

Let $R = \begin{bmatrix} c & - s \\ s & c \end{bmatrix}$, where a $c=\cos(\theta)$ and $s=\sin(\theta)$ for an angle $\theta$ to be determined. Partition 4-by-4 matrices into nested 2-by-2 blocks. 

$$
\left[\begin{array}{ll}
R & 0 \\
O & I
\end{array}\right]\left[\begin{array}{ll}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{array}\right]=\left[\begin{array}{ll}
R A_{11} & R A_{12} \\
A_{41} & A_{22}
\end{array}\right]
$$



We want $RA_{11}=\begin{bmatrix} * & * \\ 0 & * \end{bmatrix}$.

Let $A_{11}=\begin{bmatrix} e & f \\ g & h \end{bmatrix}$. Then, 

$$
RA_{11}=\begin{bmatrix} ce - sg & cf - sh \\ se+cg & sf+ch\end{bmatrix}=\begin{bmatrix} * & * \\ 0 & * \end{bmatrix}
$$

We need to find $c$ and $s$ that satisfies (2,1)-entry equality, $se+cg=0$, and $c^2 + s^2 = 1$. The following always satisfies them.

$$
c = \frac{e}{\sqrt{e^2 + g^2}}, \qquad s = -\frac{g}{\sqrt{e^2 + g^2}}
$$

In [55]:
# Specify rows involved in the Givens rotation: (2,1)-entry
i, j = 0, 1

# This creates indices for submatrix
# c.f. A[[i,j], [i,j]] returns [A[i,i], A[j,j]] by fancy indexing rule
ind = np.ix_([i,j], [i,j])
A_ = A[ind]
print(A_)

# computing cosine and sine: no need to find the angle.
# c.f.: unpacking is not allowed (e, g = *A_[:,0] --> error)
e, g = A_[0,0], A_[1,0]
den = np.sqrt(e*e + g*g)
c, s = e/den, -g/den

R_ = np.array([[c, -s], [s, c]])
print(R_@A_)


[[1. 1.]
 [2. 4.]]
[[2.23606798 4.02492236]
 [0.         0.89442719]]


2.B Full matrix

In [56]:
# Only rows i, j are affected (See the block multiplication above)
A[[i,j], :] = R_@A[[i,j], :]

print(A)

[[  2.23606798   4.02492236   7.60263112  14.75804865]
 [  0.           0.89442719   2.68328157   6.26099034]
 [  3.           9.          27.          81.        ]
 [  4.          16.          64.         256.        ]]


3. Repeat for the rest of the first column

3.A Modularize the procedure

In [57]:
def givens(A, ind):
    """
    Return 2-by-2 Givens rotation that zeros an element of given matrix

    Input:
        A (array): matrix whose single element is to be zeroed.
        ind (tuple/array of int): Two row indices involved. 
    Output:
        2-by-2 rotation matrix
            If ind = (i, j) is given (i < j), then A[i, j]-entry is to be zeroed
            by left-multiplying the returned matrix.
    """
    i, j = ind
    assert i < j, "Index must be increasing"
    
    # extract submatrix; only the i-th column matters
    A_ = A[[i, j], i]
    den = np.sqrt(A_[0]*A_[0] + A_[1]*A_[1])
    c, s = A_[0]/den, -A_[1]/den
    return np.array([[c, -s], [s, c]])

# Test the function
R1 = givens(A, (0, 1))
print(R1@A[[0,1], :])
print(R1.T @ R1)

[[ 2.23606798  4.02492236  7.60263112 14.75804865]
 [ 0.          0.89442719  2.68328157  6.26099034]]
[[1. 0.]
 [0. 1.]]


3.B Apply Givens rotation to the column

Note: We need to take the first index to be $i=0$. Otherwise, the rows that are already zeroed out are messed up.

In [66]:
for i in range(1):
    for j in range(2, n):
        ind = (i, j)
        A[ind, :] = givens(A, ind) @ A[ind, :]

print(A)
    
    

[[  5.47722558  18.25741858  64.63126179 237.34644159]
 [  0.           4.54606057  26.3964807  122.45034104]
 [  0.           0.           4.00322451  31.90976056]
 [  0.           0.           0.           2.88926047]]


4. Contructing Q

Review the first three steps

2.A. Submatrix

Let $R = \begin{bmatrix} c & - s \\ s & c \end{bmatrix}$, where a $c=\cos(\theta)$ and $s=\sin(\theta)$ for an angle $\theta$ to be determined. Partition 4-by-4 matrices into nested 2-by-2 blocks. 

$$
\left[\begin{array}{ll}
R & 0 \\
O & I
\end{array}\right]\left[\begin{array}{ll}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{array}\right]=\left[\begin{array}{ll}
R A_{11} & R A_{12} \\
A_{41} & A_{22}
\end{array}\right]
$$



We want $RA_{11}=\begin{bmatrix} * & * \\ 0 & * \end{bmatrix}$.

Let $A_{11}=\begin{bmatrix} e & f \\ g & h \end{bmatrix}$. Then, 

$$
RA_{11}=\begin{bmatrix} ce - sg & cf - sh \\ se+cg & sf+ch\end{bmatrix}=\begin{bmatrix} * & * \\ 0 & * \end{bmatrix}
$$

5. Apply Givens rotation to all lower triaular part.

In [65]:
R = A_ini.copy()
Q = np.eye(n)

for i in range(n-1): # last column is not needed to be zeroed.
    for j in range(i+1, n): # row index must exhaust all the way
        ind = (i, j)
        G_ = givens(R, ind)
        R[ind, :] = givens(R, ind) @ R[ind, :]
        Q[:, ind] = Q[:, ind] @ G_.T

print(R)
print(Q.T @ Q)
print(A_ini)
print(Q@R)
print(np.allclose(Q@R, A_ini))

[[  5.47722558  18.25741858  64.63126179 237.34644159]
 [  0.           4.54606057  26.3964807  122.45034104]
 [  0.           0.           4.00322451  31.90976056]
 [  0.           0.           0.           2.88926047]]
[[ 1.00000000e+00 -5.55111512e-17 -5.55111512e-17  0.00000000e+00]
 [-5.55111512e-17  1.00000000e+00 -1.38777878e-16 -1.24900090e-16]
 [-5.55111512e-17 -1.38777878e-16  1.00000000e+00 -4.85722573e-17]
 [ 0.00000000e+00 -1.24900090e-16 -4.85722573e-17  1.00000000e+00]]
[[  1.   1.   1.   1.]
 [  2.   4.   8.  16.]
 [  3.   9.  27.  81.]
 [  4.  16.  64. 256.]]
[[  1.   1.   1.   1.]
 [  2.   4.   8.  16.]
 [  3.   9.  27.  81.]
 [  4.  16.  64. 256.]]
True
